## Построения сводных графиков и сводных данных  

In [6]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import plotly.express as px

In [7]:
data = [
    {'Model': 'XGBoost', 'RMSE': 453.6970170345226, 'R2': 0.38289524210601866, 
     'MAE': 239.59144403129596, 'Time (sec)': 36.1, 'Experiment': 'CC50'},
    {'Model': 'Random Forest', 'RMSE': 436.11637588125706, 'R2': 0.4297939302469558, 
     'MAE': 232.8684192239013, 'Time (sec)': 37.04, 'Experiment': 'CC50'},
    {'Model': 'Neural Network', 'RMSE': 444.35585600050246, 'R2': 0.40804476859885574, 
     'MAE': 226.32868484640161, 'Time (sec)': 13.93, 'Experiment': 'CC50'},
    
    {'Model': 'XGBoost', 'RMSE': 464.625965390857, 'R2': 0.5836103432048649, 
     'MAE': 293.9290645038305, 'Time (sec)': 38.5, 'Experiment': 'IC50'},
    {'Model': 'Random Forest', 'RMSE': 451.4122933858691, 'R2': 0.6069572911500704, 
     'MAE': 278.1825514417777, 'Time (sec)': 36.4, 'Experiment': 'IC50'},
    {'Model': 'Neural Network', 'RMSE': 500.8051765036426, 'R2': 0.5162392838021651, 
     'MAE': 315.15490719920564, 'Time (sec)': 16.36, 'Experiment': 'IC50'},
    
    {'Model': 'XGBoost', 'RMSE': 1354.477735415758, 'R2': 0.0866588120110402, 
     'MAE': 185.27528527951281, 'Time (sec)': 48.16, 'Experiment': 'SI'},
    {'Model': 'Random Forest', 'RMSE': 1357.3601204029133, 'R2': 0.08276741977672442, 
     'MAE': 189.53613747755819, 'Time (sec)': 42.04, 'Experiment': 'SI'},
    {'Model': 'Neural Network', 'RMSE': 1421.3971370178479, 'R2': -0.005819785806299382, 
     'MAE': 184.91516004662247, 'Time (sec)': 5.56, 'Experiment': 'SI'}
]

df = pd.DataFrame(data)

fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "xy"}, {"type": "polar"}],
           [{"type": "xy"}, {"type": "xy"}]],
    subplot_titles=("Сравнение метрик по экспериментам", 
                   "Радиальная диаграмма (По всем экспериментам)",
                   "Время vs Точность (R²)",
                   "Сравнение времени выполнения")
)


metrics = ['RMSE', 'MAE', 'R2']
for i, metric in enumerate(metrics):
    for exp in df['Experiment'].unique():
        exp_data = df[df['Experiment'] == exp]
        fig.add_trace(
            go.Bar(
                x=exp_data['Model'],
                y=exp_data[metric],
                name=f"{metric} ({exp})",
                legendgroup=metric,
                marker_color=px.colors.qualitative.Plotly[i],
                showlegend=True
            ),
            row=1, col=1
        )


avg_metrics = df.groupby('Model').mean(numeric_only=True).reset_index()

for model in avg_metrics['Model'].unique():
    model_data = avg_metrics[avg_metrics['Model'] == model].iloc[0]
    fig.add_trace(
        go.Scatterpolar(
            r=[model_data['RMSE'], model_data['MAE'], model_data['R2'], model_data['Time (sec)']],
            theta=['RMSE', 'MAE', 'R²', 'Time'],
            fill='toself',
            name=model,
            subplot="polar"
        ),
        row=1, col=2
    )


for model in df['Model'].unique():
    model_data = df[df['Model'] == model]
    fig.add_trace(
        go.Scatter(
            x=model_data['Time (sec)'],
            y=model_data['R2'],
            mode='markers',
            marker_size=model_data['MAE']/10,
            name=model,
            text=model_data['Experiment'],
            hovertemplate="<b>%{text}</b><br>Time: %{x}s<br>R²: %{y}<br>MAE: %{marker.size:.1f}",
            showlegend=False
        ),
        row=2, col=1
    )

for exp in df['Experiment'].unique():
    exp_data = df[df['Experiment'] == exp]
    fig.add_trace(
        go.Bar(
            x=exp_data['Model'],
            y=exp_data['Time (sec)'],
            name=exp,
            showlegend=True
        ),
        row=2, col=2
    )

fig.update_layout(
    height=900,
    width=1200,
    title_text="Комплексный анализ моделей машинного обучения",
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1500]
        )
    ),
    bargap=0.15,
    bargroupgap=0.1
)

fig.update_yaxes(title_text="Значение метрики", row=1, col=1)
fig.update_yaxes(title_text="R²", range=[-0.2, 0.7], row=2, col=1)
fig.update_xaxes(title_text="Модель", row=1, col=1)
fig.update_xaxes(title_text="Время (сек)", row=2, col=1)
fig.update_xaxes(title_text="Модель", row=2, col=2)
fig.update_yaxes(title_text="Время (сек)", row=2, col=2)

fig.show()

### Общий вывод по трем экспериментам

**1. Сравнение производительности моделей**  
Во всех трех экспериментах **Random Forest** показал стабильно хорошие результаты:  
- В первых двух тестах он лидировал по R² (0.43 и 0.61) и RMSE (436.1 и 451.4)  
- В третьем тесте все модели работали плохо, но RF сохранил относительное преимущество  
- Продемонстрировал лучший баланс между точностью и стабильностью  

**XGBoost** занял второе место:  
- В первых двух экспериментах уступал Random Forest на 4-12% по R²  
- В третьем тесте показал такие же слабые результаты, как и другие модели  
- Время обучения обычно больше, чем у Random Forest  

**Neural Network** показал нестабильные результаты:  
- Быстрое обучение (в 2-8 раз быстрее конкурентов)  
- Хорошие результаты во втором тесте (R²=0.52), но провал в третьем (R²=-0.006)  
- Требует тщательной настройки архитектуры  

In [8]:

data1 = [
    {'Model': 'XGBoost', 'RMSE': 453.6970170345226, 'R2': 0.38289524210601866, 'MAE': 239.59144403129596, 'Best Params': {'subsample': 0.9, 'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.01, 'colsample_bytree': 1.0}, 'Time (sec)': 36.1},
    {'Model': 'Random Forest', 'RMSE': 436.11637588125706, 'R2': 0.4297939302469558, 'MAE': 232.8684192239013, 'Best Params': {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': 10, 'bootstrap': False}, 'Time (sec)': 37.04},
    {'Model': 'Neural Network', 'RMSE': 444.35585600050246, 'R2': 0.40804476859885574, 'MAE': 226.32868484640161, 'Best Params': {'learning_rate_init': 0.001, 'hidden_layer_sizes': (128, 64), 'batch_size': 32, 'alpha': 0.001}, 'Time (sec)': 13.93}
]

data2 = [
    {'Model': 'XGBoost', 'RMSE': 464.625965390857, 'R2': 0.5836103432048649, 'MAE': 293.9290645038305, 'Best Params': {'subsample': 0.8, 'n_estimators': 300, 'max_depth': 7, 'learning_rate': 0.01, 'colsample_bytree': 0.8}, 'Time (sec)': 38.5},
    {'Model': 'Random Forest', 'RMSE': 451.4122933858691, 'R2': 0.6069572911500704, 'MAE': 278.1825514417777, 'Best Params': {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 20, 'bootstrap': False}, 'Time (sec)': 36.4},
    {'Model': 'Neural Network', 'RMSE': 500.8051765036426, 'R2': 0.5162392838021651, 'MAE': 315.15490719920564, 'Best Params': {'learning_rate_init': 0.001, 'hidden_layer_sizes': (64, 32), 'batch_size': 128, 'alpha': 0.0001}, 'Time (sec)': 16.36}
]

data3 = [
    {'Model': 'XGBoost', 'RMSE': 1354.477735415758, 'R2': 0.0866588120110402, 'MAE': 185.27528527951281, 'Best Params': {'subsample': 1.0, 'n_estimators': 300, 'max_depth': 5, 'learning_rate': 0.2, 'colsample_bytree': 0.9}, 'Time (sec)': 48.16},
    {'Model': 'Random Forest', 'RMSE': 1357.3601204029133, 'R2': 0.08276741977672442, 'MAE': 189.53613747755819, 'Best Params': {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_depth': 20, 'bootstrap': False}, 'Time (sec)': 42.04},
    {'Model': 'Neural Network', 'RMSE': 1421.3971370178479, 'R2': -0.005819785806299382, 'MAE': 184.91516004662247, 'Best Params': {'learning_rate_init': 0.001, 'hidden_layer_sizes': (64, 32), 'batch_size': 32, 'alpha': 0.0001}, 'Time (sec)': 5.56}
]


df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)
df3 = pd.DataFrame(data3)


df1['Dataset'] = 'IC50'
df2['Dataset'] = 'CC50'
df3['Dataset'] = 'SI'


combined_df = pd.concat([df1, df2, df3], ignore_index=True)

combined_df = combined_df[['Dataset', 'Model', 'RMSE', 'R2', 'MAE', 'Time (sec)', 'Best Params']]
combined_df


,Dataset,Model,RMSE,R2,MAE,Time (sec),Best Params
0,IC50,XGBoost,453.697017,0.382895,239.591444,36.10,"{'subsample': 0.9, 'n_estimators': 200, 'max_d..."
1,IC50,Random Forest,436.116376,0.429794,232.868419,37.04,"{'n_estimators': 100, 'min_samples_split': 10,..."
2,IC50,Neural Network,444.355856,0.408045,226.328685,13.93,"{'learning_rate_init': 0.001, 'hidden_layer_si..."
3,CC50,XGBoost,464.625965,0.583610,293.929065,38.50,"{'subsample': 0.8, 'n_estimators': 300, 'max_d..."
4,CC50,Random Forest,451.412293,0.606957,278.182551,36.40,"{'n_estimators': 100, 'min_samples_split': 5, ..."
5,CC50,Neural Network,500.805177,0.516239,315.154907,16.36,"{'learning_rate_init': 0.001, 'hidden_layer_si..."
6,SI,XGBoost,1354.477735,0.086659,185.275285,48.16,"{'subsample': 1.0, 'n_estimators': 300, 'max_d..."
7,SI,Random Forest,1357.360120,0.082767,189.536137,42.04,"{'n_estimators': 100, 'min_samples_split': 10,..."
8,SI,Neural Network,1421.397137,-0.005820,184.915160,5.56,"{'learning_rate_init': 0.001, 'hidden_layer_si..."


In [9]:

data = [
    {'Model': 'Random Forest', 'Accuracy': 0.716, 'Precision': 0.697, 'Recall': 0.76, 
     'F1': 0.727, 'ROC AUC': 0.833, 'Time (sec)': 22.05, 'Experiment': 'CC50'},
    {'Model': 'K-Neighbors (GPU)', 'Accuracy': 0.721, 'Precision': 0.686, 'Recall': 0.81, 
     'F1': 0.743, 'ROC AUC': 0.827, 'Time (sec)': 10.26, 'Experiment': 'CC50'},
    {'Model': 'XGBoost', 'Accuracy': 0.697, 'Precision': 0.667, 'Recall': 0.78, 
     'F1': 0.719, 'ROC AUC': 0.815, 'Time (sec)': 31.1, 'Experiment': 'CC50'},
    
    {'Model': 'XGBoost', 'Accuracy': 0.667, 'Precision': 0.643, 'Recall': 0.74, 
     'F1': 0.688, 'ROC AUC': 0.760, 'Time (sec)': 34.24, 'Experiment': 'IC50'},
    {'Model': 'K-Neighbors (GPU)', 'Accuracy': 0.667, 'Precision': 0.651, 'Recall': 0.71, 
     'F1': 0.679, 'ROC AUC': 0.747, 'Time (sec)': 10.25, 'Experiment': 'IC50'},
    {'Model': 'Random Forest', 'Accuracy': 0.697, 'Precision': 0.664, 'Recall': 0.79, 
     'F1': 0.721, 'ROC AUC': 0.778, 'Time (sec)': 22.38, 'Experiment': 'IC50'},
    
    {'Model': 'XGBoost', 'Accuracy': 0.667, 'Precision': 0.681, 'Recall': 0.62, 
     'F1': 0.649, 'ROC AUC': 0.694, 'Time (sec)': 36.59, 'Experiment': 'SI'},
    {'Model': 'K-Neighbors (GPU)', 'Accuracy': 0.637, 'Precision': 0.631, 'Recall': 0.65, 
     'F1': 0.640, 'ROC AUC': 0.713, 'Time (sec)': 13.28, 'Experiment': 'SI'},
    {'Model': 'Random Forest', 'Accuracy': 0.647, 'Precision': 0.667, 'Recall': 0.58, 
     'F1': 0.620, 'ROC AUC': 0.680, 'Time (sec)': 18.73, 'Experiment': 'SI'},
    
    {'Model': 'XGBoost', 'Accuracy': 0.741, 'Precision': 0.700, 'Recall': 0.486, 
     'F1': 0.574, 'ROC AUC': 0.763, 'Time (sec)': 34.62, 'Experiment': 'SI_8'},
    {'Model': 'K-Neighbors (GPU)', 'Accuracy': 0.657, 'Precision': 0.522, 'Recall': 0.486, 
     'F1': 0.504, 'ROC AUC': 0.724, 'Time (sec)': 10.66, 'Experiment': 'SI_8'},
    {'Model': 'Random Forest', 'Accuracy': 0.716, 'Precision': 0.623, 'Recall': 0.528, 
     'F1': 0.571, 'ROC AUC': 0.750, 'Time (sec)': 22.35, 'Experiment': 'SI_8'}
]

df = pd.DataFrame(data)

fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "xy"}, {"type": "polar"}],
           [{"type": "xy"}, {"type": "xy"}]],
    subplot_titles=("Сравнение метрик по экспериментам", 
                   "Радиальная диаграмма (Усредненные метрики)",
                   "Время vs ROC AUC",
                   "Сравнение времени выполнения")
)


metrics = ['Accuracy', 'Precision', 'Recall', 'F1', 'ROC AUC']
for i, metric in enumerate(metrics[:4]):
    for exp in df['Experiment'].unique():
        exp_data = df[df['Experiment'] == exp]
        fig.add_trace(
            go.Bar(
                x=exp_data['Model'],
                y=exp_data[metric],
                name=f"{metric} ({exp})",
                legendgroup=metric,
                marker_color=px.colors.qualitative.Plotly[i],
                showlegend=True
            ),
            row=1, col=1
        )

avg_metrics = df.groupby('Model').mean(numeric_only=True).reset_index()

for model in avg_metrics['Model'].unique():
    model_data = avg_metrics[avg_metrics['Model'] == model].iloc[0]
    fig.add_trace(
        go.Scatterpolar(
            r=[model_data['Accuracy'], model_data['Precision'], 
               model_data['Recall'], model_data['F1'], model_data['ROC AUC']],
            theta=['Accuracy', 'Precision', 'Recall', 'F1', 'ROC AUC'],
            fill='toself',
            name=model,
            subplot="polar"
        ),
        row=1, col=2
    )

for model in df['Model'].unique():
    model_data = df[df['Model'] == model]
    fig.add_trace(
        go.Scatter(
            x=model_data['Time (sec)'],
            y=model_data['ROC AUC'],
            mode='markers',
            marker_size=model_data['Accuracy']*50,
            name=model,
            text=model_data['Experiment'],
            hovertemplate="<b>%{text}</b><br>Time: %{x}s<br>ROC AUC: %{y}<br>Accuracy: %{marker.size:.1f}%",
            showlegend=False
        ),
        row=2, col=1
    )

for exp in df['Experiment'].unique():
    exp_data = df[df['Experiment'] == exp]
    fig.add_trace(
        go.Bar(
            x=exp_data['Model'],
            y=exp_data['Time (sec)'],
            name=exp,
            showlegend=True
        ),
        row=2, col=2
    )

fig.update_layout(
    height=900,
    width=1200,
    title_text="Комплексный анализ классификационных моделей",
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1]
        )
    ),
    bargap=0.15,
    bargroupgap=0.1
)

fig.update_yaxes(title_text="Значение метрики", range=[0, 1], row=1, col=1)
fig.update_yaxes(title_text="ROC AUC", range=[0.5, 1], row=2, col=1)
fig.update_xaxes(title_text="Модель", row=1, col=1)
fig.update_xaxes(title_text="Время (сек)", row=2, col=1)
fig.update_xaxes(title_text="Модель", row=2, col=2)
fig.update_yaxes(title_text="Время (сек)", row=2, col=2)

fig.show()
